In [21]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
import finplot as fplt

import polygon_backfill as pb
import market_cluster as mc
import s3_datasets as s3d
import filters as ft
import local_backfill as lb
import bars as sb
import bar_labels as bl
import load_ticks as lt

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# import ipdb
# ipdb.set_trace(context=10)

symbol='GLD'
date='2020-08-27'
tick_type='trades'
result_path='/Users/bobcolner/QuantClarity/pandas-polygon/data'
    
# .plot_bokeh(kind='hist', bins=50, sizing_mode="scale_height", disable_scientific_axes=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)
# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')
# sdf = mdf[mdf.symbol=='SPY']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
s3d.list_symbol(symbol)

['polygon-equities/data/trades/symbol=GLD/date=2020-10-01',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-02',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-05',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-06',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-07',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-08',
 'polygon-equities/data/trades/symbol=GLD/date=2020-10-09']

In [31]:
df = lt.load_ticks('/Users/bobcolner/QuantClarity/pandas-polygon/data/', symbol='GLD', date='2020-10-08')
df

trying to get ticks from local file...
trying to get ticks from s3...
trying to get ticks from polygon API
Downloaded:  47226 GLD ticks; latest time(NYC):  2020-10-08 20:00:00.002238464-04:00
dropped 202 ticks ( 0.43 %)


,date_time,price,volume
0,2020-10-08 08:00:00.011162880,177.500000,221
1,2020-10-08 08:04:41.825920768,177.550003,1
2,2020-10-08 08:04:47.608566272,177.559998,100
3,2020-10-08 08:04:47.710558976,177.559998,100
4,2020-10-08 08:04:47.812080384,177.559998,100
...,...,...,...
47019,2020-10-08 23:57:15.321619968,177.940002,2
47020,2020-10-08 23:57:16.705828864,177.940002,2
47021,2020-10-08 23:58:44.969093888,177.949997,22
47022,2020-10-08 23:58:44.971233536,177.949997,3


In [ ]:
# load ticks (local or api)
ticks_df = lb.load_ticks(symbol, date, result_path)

# sample ticks -> bars
renko_size = 0.2
renko_reversal = 2
thresh = {
    # GT thresholds
    'renko_size': renko_size,
    'renko_reveral_multiple': renko_reversal,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 5,
    'min_tick_count': 10,
}

tdf = ticks_df[(ticks_df.date_time.dt.hour >= 13) & (ticks_df.date_time.dt.hour <= 22)]
bars, _ = sb.build_bars(ticks_df=tdf, thresh=thresh)
len(bars)

In [ ]:
lbars_df.tick_imbalance_run.hist(bins=50)

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

# lbars = bl.label_bars_triple_barrier(bars, ticks_df, risk_level, label_horizon_mins, reward_base)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

In [ ]:
lbars_df.to_feather('lbars_df.feather')

In [ ]:
lbars_df[['price_wmean', 'price_close']].plot(subplots=False)

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])